### Download Illumina sequence fastq files from Owl 

In [ ]:
# download using bash script
! bash ../scripts/download-quantseq.sh

### Install tag-seq scripts
I opted to install these scripts into my bioinformatics directory, where I house these types of programs. Create a {tagseq} variable

In [ ]:
! git clone https://github.com/z0on/tag-based_RNAseq.git /Applications/bioinformatics/tag-based_RNAseq/

In [5]:
# create path variable to blast directory 
tagseq = "/Applications/bioinformatics/tag-based_RNAseq/"

In [1]:
! ls ../data/illumina-raw/

CP-KS-LibL-L10-G_S59_L006_R1_001.fastq
CP-KS-LibL-L11-G_S65_L006_R1_001.fastq
CP-KS-LibL-L13-G_S60_L006_R1_001.fastq
CP-KS-LibL-L14-G_S63_L006_R1_001.fastq
CP-KS-LibL-L15-G_S68_L006_R1_001.fastq
CP-KS-LibL-L19-G_S67_L006_R1_001.fastq
CP-KS-LibL-L2-G_S34_L006_R1_001.fastq
CP-KS-LibL-L21-G_S58_L006_R1_001.fastq
CP-KS-LibL-L23-G_S61_L006_R1_001.fastq
CP-KS-LibL-L24-G_S62_L006_R1_001.fastq
CP-KS-LibL-L29-G_S36_L006_R1_001.fastq
CP-KS-LibL-L3-G_S64_L006_R1_001.fastq
CP-KS-LibL-L32-G_S35_L006_R1_001.fastq
CP-KS-LibL-L33-G_S66_L006_R1_001.fastq
CP-KS-LibL-L34-L_S25_L006_R1_001.fastq
CP-KS-LibL-L35-L_S31_L006_R1_001.fastq
CP-KS-LibL-L37-L_S32_L006_R1_001.fastq
CP-KS-LibL-L39-L_S29_L006_R1_001.fastq
CP-KS-LibL-L41-L_S27_L006_R1_001.fastq
CP-KS-LibL-L44-L_S30_L006_R1_001.fastq
CP-KS-LibL-L45-L_S28_L006_R1_001.fastq
CP-KS-LibL-L47-L_S33_L006_R1_001.fastq
CP-KS-LibL-L7-G_S57_L006_R1_001.fastq
CP-KS-LibL-SN1016-G_S26_L006_R1_001.fastq
clean
maps
quatseq-files.txt
tagSeq_trim.sh


### Adaptor trimming, deduplicating, and quality filtering:
Creating and launching the cleaning process for all files in the same time:

In [86]:
! ls {tagseq}

2bRAD_bowtie2_launch.pl        p5barcodes_order.xlsx
README.md                      picogreen.R
RNASeq_sample_prep_dec2017.pdf picogreen.csv
TagSeq_GSAF_Price.xlsx         rnaseq_clipper.pl
TagSeq_sample_prep_dec2017.pdf rnaseq_clipper0.pl
TagSeq_sample_prep_may2018.pdf rnaseq_clipper_fasta.pl
countreads.pl                  rnaseq_clipper_old.pl
dna.mixing.R                   samcount.pl
dupCount.R                     samcount_launch.pl
expression_compiler.pl         samcount_launch_bt2.pl
iRNAseq_bowtie2map.pl          samcount_v.0.1.pl
iRNAseq_shrimpmap_SAM.pl       selectFastaByHeader.pl
iRNAseq_trim_launch.pl         splitFastaByHeader.pl
iRNAseq_trim_launch0.pl        tag-seq_scripts_manual.pdf
iRNAseq_trim_launch_bgi.pl     tagSeq_processing_README.txt
illumina_mix_data.csv          tagseq_bowtie2map.pl
isogroup_namer.pl              tagseq_clipper.pl
launcher_creator.py            tagseq_oligo_order.xls
mix_illumina_qpcr.R            tagseq_trim_launch.pl
ngs_concat.pl          

In [87]:
! cat {tagseq}tagseq_clipper.pl

#!/usr/bin/perl

$usage= "

tagseq_clipper.pl  : 

Clips 5'-leader off Illumina fastq reads in RNA-seq

Removes duplicated reads sharing the same degenerate header and 
the first 20 bases of the sequence (reads containing N bases in this
region are discarded, too)

prints to STDOUT

arguments:
1 : fastq file name
2 : string to define the leading sequence, default '[ATGC]?[ATGC][AC][AT]GGG+|[ATGC]?[ATGC]TGC[AC][AT]GGG+|[ATGC]?[ATGC]GC[AT]TC[ACT][AC][AT]GGG+'
'keep' : optional flag to say whether the sequences without leader should be kept. 
		 By default, they are discarded.

Example:
tagseq_clipper.pl D6.fastq

					 
";

my $fq=shift or die $usage;
my $lead="";
my $keep=0;
if ($ARGV[0]) { $lead=$ARGV[0];}
else { $lead="[ATGC]?[ATGC][AC][AT]GGG+|[ATGC]?[ATGC]TGC[AC][AT]GGG+|[ATGC]?[ATGC]GC[AT]TC[ACT][AC][AT]GGG+";}
if ($ARGV[1]) {$keep=1;}
my $trim=0;
my $name="";
my $name2="";
my $seq="";
my $qua="";
my %seen={};
open INP, $fq or die "cannot open file $fq\n";

my $nohead=0;
my $dups=0

In [88]:
# test tagseq shortcut and perl script 
! cat {tagseq}tagseq_trim_launch.pl

#!/usr/bin/perl

my $usage= "

tagseq_trim_launch.pl :

Prints out a series of commands to trim Illumina tag-seq reads, one for each reads file
(tagseq_clipper.pl | fasx_clipper | fastx_quality_filter )

prints to STDOUT

Arguments:
1: glob to fastq files
2: optional, the position of name-deriving string in the file name
	if separated by underscores, 
	such as: input file Sample_RNA_2DVH_L002_R1.cat.fastq
	specifying arg2 as '3' would create output file with a name '2DVH.fastq'	

Example: 
tagseq_trim_launch.pl '\.fq$' > clean

NOTE: use this script if your qualities in the fastq files are 33-based;
if not, use tagseq_trim_launch_bgi.pl instead

";

if (!$ARGV[0]) { die $usage;}
my $glob=$ARGV[0];

opendir THIS, ".";
my @fqs=grep /$glob/,readdir THIS;
my $outname="";

foreach $fqf (@fqs) {
	if ($ARGV[1]) {
		my @parts=split('_',$fqf);
		$outname=$parts[$ARGV[1]-1].".trim";
	}
	else { $outname=$fqf.".trim";}
	print "tagseq_clipper.pl $fqf | fastx_clipper -a AAAAAAAA -l 20 -Q33 | fastx_c

In [89]:
# test fast_clipper 
! fastx_clipper -h 

usage: fastx_clipper [-h] [-a ADAPTER] [-D] [-l N] [-n] [-d N] [-c] [-C] [-o] [-v] [-z] [-i INFILE] [-o OUTFILE]
Part of FASTX Toolkit 0.0.14 by A. Gordon (assafgordon@gmail.com)

   [-h]         = This helpful help screen.
   [-a ADAPTER] = ADAPTER string. default is CCTTAAGG (dummy adapter).
   [-l N]       = discard sequences shorter than N nucleotides. default is 5.
   [-d N]       = Keep the adapter and N bases after it.
                  (using '-d 0' is the same as not using '-d' at all. which is the default).
   [-c]         = Discard non-clipped sequences (i.e. - keep only sequences which contained the adapter).
   [-C]         = Discard clipped sequences (i.e. - keep only sequences which did not contained the adapter).
   [-k]         = Report Adapter-Only sequences.
   [-n]         = keep sequences with unknown (N) nucleotides. default is to discard such sequences.
   [-v]         = Verbose - report number of sequences.
                  If [-o] is specified,  report will be

In [90]:
# test fastq_quality_filter
! fastq_quality_filter -h

usage: fastq_quality_filter [-h] [-v] [-q N] [-p N] [-z] [-i INFILE] [-o OUTFILE]
Part of FASTX Toolkit 0.0.14 by A. Gordon (assafgordon@gmail.com)

   [-h]         = This helpful help screen.
   [-q N]       = Minimum quality score to keep.
   [-p N]       = Minimum percent of bases that must have [-q] quality.
   [-z]         = Compress output with GZIP.
   [-i INFILE]  = FASTA/Q input file. default is STDIN.
   [-o OUTFILE] = FASTA/Q output file. default is STDOUT.
   [-v]         = Verbose - report number of sequences.
                  If [-o] is specified,  report will be printed to STDOUT.
                  If [-o] is not specified (and output goes to STDOUT),
                  report will be printed to STDERR.



In [3]:
cd ../data/illumina-raw/

/Users/laura/Documents/roberts-lab/laura-quantseq/data/illumina-raw


In [78]:
# run perl script that creates list of executables for each fastq file 
! {tagseq}tagseq_trim_launch.pl '\.fastq$' > clean

In [4]:
! head -2 clean

tagseq_clipper.pl CP-KS-LibL-L21-G_S58_L006_R1_001.fastq | fastx_clipper -a AAAAAAAA -l 20 -Q33 | fastx_clipper -a AGATCGGAAG -l 20 -Q33 | fastq_quality_filter -Q33 -q 20 -p 90 >CP-KS-LibL-L21-G_S58_L006_R1_001.fastq.trim
tagseq_clipper.pl CP-KS-LibL-L23-G_S61_L006_R1_001.fastq | fastx_clipper -a AAAAAAAA -l 20 -Q33 | fastx_clipper -a AGATCGGAAG -l 20 -Q33 | fastq_quality_filter -Q33 -q 20 -p 90 >CP-KS-LibL-L23-G_S61_L006_R1_001.fastq.trim


### Based on repeated functions in the clean file, I wrote a script to loop through the files and execute all commands 

In [82]:
! ls

CP-KS-LibL-L10-G_S59_L006_R1_001.fastq
CP-KS-LibL-L11-G_S65_L006_R1_001.fastq
CP-KS-LibL-L13-G_S60_L006_R1_001.fastq
CP-KS-LibL-L14-G_S63_L006_R1_001.fastq
CP-KS-LibL-L15-G_S68_L006_R1_001.fastq
CP-KS-LibL-L19-G_S67_L006_R1_001.fastq
CP-KS-LibL-L2-G_S34_L006_R1_001.fastq
CP-KS-LibL-L21-G_S58_L006_R1_001.fastq
CP-KS-LibL-L23-G_S61_L006_R1_001.fastq
CP-KS-LibL-L24-G_S62_L006_R1_001.fastq
CP-KS-LibL-L29-G_S36_L006_R1_001.fastq
CP-KS-LibL-L3-G_S64_L006_R1_001.fastq
CP-KS-LibL-L32-G_S35_L006_R1_001.fastq
CP-KS-LibL-L33-G_S66_L006_R1_001.fastq
CP-KS-LibL-L34-L_S25_L006_R1_001.fastq
CP-KS-LibL-L35-L_S31_L006_R1_001.fastq
CP-KS-LibL-L37-L_S32_L006_R1_001.fastq
CP-KS-LibL-L39-L_S29_L006_R1_001.fastq
CP-KS-LibL-L41-L_S27_L006_R1_001.fastq
CP-KS-LibL-L44-L_S30_L006_R1_001.fastq
CP-KS-LibL-L45-L_S28_L006_R1_001.fastq
CP-KS-LibL-L47-L_S33_L006_R1_001.fastq
CP-KS-LibL-L7-G_S57_L006_R1_001.fastq
CP-KS-LibL-SN1016-G_S26_L006_R1_001.fastq
clean
quatseq-files.txt


Can't use the global variable {tagseq} inside my for loop ... figure out why. 

In [129]:
%%bash 

for file in *.fastq
do
#strip .fastq and directorys tructure from each file, then
# add suffice .fastq.trim to create output name for each file
results_file="$(basename -a $file).trim"

# run tagseq scripts on each file
/Applications/bioinformatics/tag-based_RNAseq/tagseq_clipper.pl $file | \
fastx_clipper -a AAAAAAAA -l 20 -Q33 | \
fastx_clipper -a AGATCGGAAG -l 20 -Q33 | \
fastq_quality_filter -Q33 -q 20 -p 90 >\
../tagseq_trim/$results_file 
done

CP-KS-LibL-L10-G_S59_L006_R1_001.fastq	total:6452865	goods:27350	dups:135439	noheader:6290063	N.in.header:13
CP-KS-LibL-L11-G_S65_L006_R1_001.fastq	total:5761774	goods:27135	dups:112294	noheader:5622331	N.in.header:14
CP-KS-LibL-L13-G_S60_L006_R1_001.fastq	total:4723646	goods:3787	dups:63418	noheader:4656437	N.in.header:4
CP-KS-LibL-L14-G_S63_L006_R1_001.fastq	total:5906432	goods:26663	dups:105949	noheader:5773809	N.in.header:11
CP-KS-LibL-L15-G_S68_L006_R1_001.fastq	total:9442881	goods:30660	dups:90323	noheader:9321885	N.in.header:13
CP-KS-LibL-L19-G_S67_L006_R1_001.fastq	total:6377544	goods:28528	dups:88628	noheader:6260379	N.in.header:9
CP-KS-LibL-L2-G_S34_L006_R1_001.fastq	total:5636566	goods:20425	dups:96398	noheader:5519737	N.in.header:6
CP-KS-LibL-L21-G_S58_L006_R1_001.fastq	total:4073358	goods:18890	dups:66504	noheader:3987951	N.in.header:13
CP-KS-LibL-L23-G_S61_L006_R1_001.fastq	total:6291394	goods:24916	dups:107619	noheader:6158846	N.in.header:13
CP-KS-LibL-L24-G_S62_L006_R1_

### download and format reference transcriptome for read alignment 

In [138]:
%%bash
# if needed, download the assembled transcriptome

cd ../

curl \
http://owl.fish.washington.edu/generosa/2018-Oly-gonad-assembly/20181029_Trinity/Trinity.fasta \
> ../results/trinity/20181029-olurida-gonad-assembly.fasta

curl \
http://owl.fish.washington.edu/generosa/2018-Oly-gonad-assembly/20181029_Trinity/Trinity.fasta.shasum.txt \
> ../results/trinity/20181029-olurida-gonad-assembly-shasum.txt

# shasum of trinity fasta file, transferred directly from Mox
cat ../results/trinity/20181029-olurida-gonad-assembly-shasum.txt

# Compare transcritome fasta file on ostrich to the original on mox using shasum 
shasum ../results/trinity/20181029-olurida-gonad-assembly.fasta

06c996f75944389ce5d7d2bce64f278fbe27b208  Trinity.fasta
06c996f75944389ce5d7d2bce64f278fbe27b208  ../results/trinity/20181029-olurida-gonad-assembly.fasta


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  162M  100  162M    0     0  1208k      0  0:02:18  0:02:18 --:--:-- 1681k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    56  100    56    0     0    366      0 --:--:-- --:--:-- --:--:--   368


In [139]:
! bowtie2-build -h

Bowtie 2 version 2.3.4.3 by Ben Langmead (langmea@cs.jhu.edu, www.cs.jhu.edu/~langmea)
Usage: bowtie2-build [options]* <reference_in> <bt2_index_base>
    reference_in            comma-separated list of files with ref sequences
    bt2_index_base          write bt2 data to files with this dir/basename
*** Bowtie 2 indexes work only with v2 (not v1).  Likewise for v1 indexes. ***
Options:
    -f                      reference files are Fasta (default)
    -c                      reference sequences given on cmd line (as
                            <reference_in>)
    --large-index           force generated index to be 'large', even if ref
                            has fewer than 4 billion nucleotides
    --debug                 use the debug binary; slower, assertions enabled
    --sanitized             use sanitized binary; slower, uses ASan and/or UBSan
    --verbose               log the issued command
    -a/--noauto             disable automatic -p/--bmax/--dcv memory-fitting
   

In [109]:
cd ../../results/trinity/

/Users/laura/Documents/roberts-lab/laura-quantseq/results/trinity


In [110]:
%%bash
### creating bowtie2 index for your transcriptome:

bowtie2-build \
20181029-olurida-gonad-assembly_clust.fasta \
20181029-olurida-gonad-assembly_clust.fasta

Settings:
  Output files: "20181029-olurida-gonad-assembly_clust.fasta.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  20181029-olurida-gonad-assembly_clust.fasta
Reading reference sizes
  Time reading reference sizes: 00:00:01
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
bmax according to bmaxDivN setting: 24095929
Using parameters --bmax 18071947 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 1807

Building a SMALL index


In [91]:
! ls *.bt2

20181029-olurida-gonad-assembly-shasum.txt
20181029-olurida-gonad-assembly.fasta
20181029-olurida-gonad-assembly.fasta.1.bt2
20181029-olurida-gonad-assembly.fasta.2.bt2
20181029-olurida-gonad-assembly.fasta.3.bt2
20181029-olurida-gonad-assembly.fasta.4.bt2
20181029-olurida-gonad-assembly.fasta.rev.1.bt2
20181029-olurida-gonad-assembly.fasta.rev.2.bt2
20181029-olurida-gonad-assembly_clust.fasta
20181029-olurida-gonad-assembly_clust.fasta.clstr
20181029-olurida-gonad-assembly_clust_iso.fasta
20181029-olurida-gonad-assembly_clust_iso_seq2iso.tab
20181029-olurida-gonad-assembly_iso.fasta
20181029-olurida-gonad-assembly_iso_seq2iso.tab
20181029-olurida-gonad-assembly_seq2iso.tab


## Mapping reads to transcriptome

Change directory to where trimmed quantseq files are located

In [92]:
cd ../tagseq_trim/

/Users/laura/Documents/roberts-lab/laura-quantseq/data/tagseq_trim


In [98]:
! {tagseq}tagseq_bowtie2map.pl "trim$" \
../../results/trinity/20181029-olurida-gonad-assembly_clust.fasta \
> maps-transc

In [99]:
! head -2 maps-transc

bowtie2 --local -x ../../results/trinity/20181029-olurida-gonad-assembly_clust.fasta -U CP-KS-LibL-L37-L_S32_L006_R1_001.fastq.trim -S CP-KS-LibL-L37-L_S32_L006_R1_001.fastq.trim.sam --no-hd --no-sq --no-unal -k 5
bowtie2 --local -x ../../results/trinity/20181029-olurida-gonad-assembly_clust.fasta -U CP-KS-LibL-L3-G_S64_L006_R1_001.fastq.trim -S CP-KS-LibL-L3-G_S64_L006_R1_001.fastq.trim.sam --no-hd --no-sq --no-unal -k 5


###  Use a loop to execute the commands written in 'maps'; redirect screen output/error in bowtieout.txt

In [104]:
pwd

'/Users/laura/Documents/roberts-lab/laura-quantseq/data/tagseq_trim'

--no-unal  = suppress SAM records for unaligned reads  
--no-sq    = suppress @SQ header lines  
--no-hd    = suppress SAM header lines (starting with @)  
-k 5       = report up to <5> aligns per read  

In [105]:
%%bash 

for file in *.trim
do
#strip .fastq and directorys tructure from each file, then
# add suffice .fastq.trim to create output name for each file
results_file="$(basename -a $file).sam"

# run tagseq scripts on each file
bowtie2 --local -x \
../../results/trinity/20181029-olurida-gonad-assembly_clust.fasta \
-U $file \
-S $results_file \
--no-unal -k 5; \
done >> bowtieout-transc.txt 2>&1

In [12]:
! cat bowtieout.txt

21874 reads; of these:
  21874 (100.00%) were unpaired; of these:
    2825 (12.91%) aligned 0 times
    6949 (31.77%) aligned exactly 1 time
    12100 (55.32%) aligned >1 times
87.09% overall alignment rate
21768 reads; of these:
  21768 (100.00%) were unpaired; of these:
    3501 (16.08%) aligned 0 times
    6939 (31.88%) aligned exactly 1 time
    11328 (52.04%) aligned >1 times
83.92% overall alignment rate
2649 reads; of these:
  2649 (100.00%) were unpaired; of these:
    1883 (71.08%) aligned 0 times
    252 (9.51%) aligned exactly 1 time
    514 (19.40%) aligned >1 times
28.92% overall alignment rate
21489 reads; of these:
  21489 (100.00%) were unpaired; of these:
    3032 (14.11%) aligned 0 times
    6776 (31.53%) aligned exactly 1 time
    11681 (54.36%) aligned >1 times
85.89% overall alignment rate
23903 reads; of these:
  23903 (100.00%) were unpaired; of these:
    4815 (20.14%) aligned 0 times
    6406 (26.80%) aligned exactly 1 time
    12682 (53.06%) aligned >1 times
7

In [14]:
# alignment rates:
! grep "overall alignment rate"  bowtieout.txt

87.09% overall alignment rate
83.92% overall alignment rate
28.92% overall alignment rate
85.89% overall alignment rate
79.86% overall alignment rate
86.02% overall alignment rate
86.38% overall alignment rate
88.04% overall alignment rate
86.00% overall alignment rate
89.57% overall alignment rate
82.48% overall alignment rate
37.29% overall alignment rate
86.37% overall alignment rate
86.27% overall alignment rate
78.36% overall alignment rate
79.96% overall alignment rate
75.70% overall alignment rate
82.03% overall alignment rate
83.13% overall alignment rate
83.36% overall alignment rate
75.97% overall alignment rate
78.80% overall alignment rate
85.48% overall alignment rate
84.32% overall alignment rate


## Generating read-counts-per gene 

First, must have a tab-delimited file giving correspondence between contigs in the transcriptome fasta file and genes. Typically, each gene is represented by several contigs in the transcriptome. To create such table for a Trinity-derived de novo transcriptome:

In [16]:
! head -4 ../../results/trinity/20181029-olurida-gonad-assembly.fasta

>TRINITY_DN88414_c0_g1_i1 len=228 path=[0:0-227]
GGCACTATTTGAGTCTGATTCTCCTCTTGATAGACGACCATGAGATCGAAATCTCTCAAACAGATCATCAGACCGTTTGCTTTGCATGCGTAAAAATGAATTCGCCCCACAAACAAAGTCCCCGAGGTTTTGTATGATCTGAGTGGAATTCATTTTCTTTTTATTCATGTCTCTGAAGAATTTTTCCATCTTATTGACATCTAACTCCTTCAAAGATGTCAAATCACT
>TRINITY_DN88428_c0_g1_i1 len=286 path=[0:0-285]
CGAAAACAGTCAGATGTAGGTATGGAAATATCCCAATCATCAGAAATCCGACTACAGGTGTAAATGATATTAGTATGTTTACACTGCATGCACCGTTAACATTGACCGATCGACGTGGAGATGGAATGGGAATACGTTCTACTGTCAATGCACACCTTGGAGCTTGATCTGGCAGTGACGAATATTTACTCAAGTAGCAGATAGATAGTTTAATGTTCAGTTTCCGTCACGGGTTTTGTTTCACTCTTACAGAGACCTTGCCCGTGGGGTCCGGGTTGGAATAGGT


In [63]:
! grep ">" ../../results/trinity/20181029-olurida-gonad-assembly.fasta | \
perl -pe 's/^>(\S+)\|(\S+)(_i\d+)\s.+/$1/' \
> ../../results/20181029-olurida-gonad-assembly_seq2iso.tab

In [62]:
! head ../../results/20181029-olurida-gonad-assembly_seq2iso.tab

>TRINITY_DN88414_c0_g1_i1 len=228 path=[0:0-227]
>TRINITY_DN88428_c0_g1_i1 len=286 path=[0:0-285]
>TRINITY_DN88485_c0_g1_i1 len=493 path=[0:0-492]
>TRINITY_DN88461_c0_g1_i1 len=305 path=[0:0-304]
>TRINITY_DN88458_c0_g1_i1 len=843 path=[0:0-842]
>TRINITY_DN88473_c0_g1_i1 len=232 path=[0:0-231]
>TRINITY_DN88442_c0_g1_i1 len=444 path=[0:0-443]
>TRINITY_DN88493_c0_g1_i1 len=865 path=[0:0-864]
>TRINITY_DN88449_c0_g1_i1 len=270 path=[0:0-269]
>TRINITY_DN88451_c0_g1_i1 len=760 path=[0:0-759]


### counting hits per isogroup:

In [10]:
pwd

'/Users/laura/Documents/roberts-lab/laura-quantseq/data/tagseq_trim'

In [8]:
! {tagseq}samcount_launch_bt2.pl '\.sam' \
../../results/20181029-olurida-gonad-assembly_seq2iso.tab > sc

In [9]:
! head -2 sc

samcount.pl CP-KS-LibL-SN1016-G_S26_L006_R1_001.fastq.trim.sam.counts ../../results/20181029-olurida-gonad-assembly_seq2iso.tab aligner=bowtie2 >CP-KS-LibL-SN1016-G_S26_L006_R1_001.fastq.trim.sam.counts.counts
samcount.pl CP-KS-LibL-L33-G_S66_L006_R1_001.fastq.trim.sam ../../results/20181029-olurida-gonad-assembly_seq2iso.tab aligner=bowtie2 >CP-KS-LibL-L33-G_S66_L006_R1_001.fastq.trim.sam.counts


In [11]:
%%bash 

for file in *.sam
do
#strip .fastq and directory s tructure from each file, then
# add suffice .fastq.trim to create output name for each file
results_file="$(basename -a $file).counts"

/Applications/bioinformatics/tag-based_RNAseq/samcount.pl $file \
../../results/20181029-olurida-gonad-assembly_seq2iso.tab \
aligner=bowtie2 mult.iso=random >\
$results_file
done >> bowtiealigner.txt 2>&1

In [12]:
! head bowtiealigner.txt

adding a count to a randomly picked isogroup when a read maps to multiple isogroups
TRINITY_DN15009_c2_g1_i1 has no isogroup designation
TRINITY_DN1226_c0_g1_i6 has no isogroup designation
TRINITY_DN1226_c0_g1_i4 has no isogroup designation
TRINITY_DN1226_c0_g1_i1 has no isogroup designation
TRINITY_DN1226_c0_g1_i5 has no isogroup designation
TRINITY_DN1226_c0_g1_i2 has no isogroup designation
TRINITY_DN15009_c2_g1_i1 has no isogroup designation
TRINITY_DN934_c0_g1_i1 has no isogroup designation
TRINITY_DN934_c0_g1_i2 has no isogroup designation


In [13]:
! {tagseq}expression_compiler.pl *.sam.counts > quantseq_allcounts.txt

In [14]:
! head quantseq_allcounts.txt

	CP-KS-LibL-L10-G_S59_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-L11-G_S65_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-L13-G_S60_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-L14-G_S63_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-L15-G_S68_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-L19-G_S67_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-L2-G_S34_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-L21-G_S58_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-L23-G_S61_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-L24-G_S62_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-L29-G_S36_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-L3-G_S64_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-L32-G_S35_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-L33-G_S66_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-L34-L_S25_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-L35-L_S31_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-L37-L_S32_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-L39-L_S29_L006_R1_001.fastq.trim.sam.counts	CP-KS-LibL-

### Try dave's revised 1-liner perl script:

In [87]:
! grep ">" ../../results/trinity/20181029-olurida-gonad-assembly.fasta | \
perl -pe 's/^>(\S+)(_i\d+)\s.+/$1$2\t$1/' \
> ../../results/20181029-olurida-gonad-assembly_seq2iso.tab

In [89]:
! grep ">" ../../results/trinity/20181029-olurida-gonad-assembly.fasta | \
perl -pe 's/^T(_g\d+)\s.+/$1/' \
> ../../results/20181029-olurida-gonad-assembly_seq2iso.tab

In [90]:
! head ../../results/20181029-olurida-gonad-assembly_seq2iso.tab

>TRINITY_DN88414_c0_g1_i1 len=228 path=[0:0-227]
>TRINITY_DN88428_c0_g1_i1 len=286 path=[0:0-285]
>TRINITY_DN88485_c0_g1_i1 len=493 path=[0:0-492]
>TRINITY_DN88461_c0_g1_i1 len=305 path=[0:0-304]
>TRINITY_DN88458_c0_g1_i1 len=843 path=[0:0-842]
>TRINITY_DN88473_c0_g1_i1 len=232 path=[0:0-231]
>TRINITY_DN88442_c0_g1_i1 len=444 path=[0:0-443]
>TRINITY_DN88493_c0_g1_i1 len=865 path=[0:0-864]
>TRINITY_DN88449_c0_g1_i1 len=270 path=[0:0-269]
>TRINITY_DN88451_c0_g1_i1 len=760 path=[0:0-759]


In [24]:
! cat {tagseq}samcount.pl

#!/usr/bin/perl

$usage="

samcount v.0.2 (November 2014):

counts reads mapping to isogrops in SAM files

Arguments:

arg1: SAM file (by cluster, contig, or isotig)
arg2: a table in the form 'reference_seq<tab>gene_ID', giving the correspondence of 
reference sequences to genes. With 454-deived transcriptome, the gene_ID would be isogroup; 
with Trinity-derived transcriptiome,it would be component.

dup.reads=keep|toss : whether to remove exact sequence-duplicate reads mapping to the 
same position in the reference. Default keep (duplicates are supposed to be tossed at the 
trimming stage).

aligner=gmapper|bowtie2 : aligner that made the SAM file. Default bowtie2.
                          bowtie2 is assumed to be used in -k mode.

mult.iso=random|toss : (for aligner=gmapper) if a read maps to multiple isogroups, it is 
disregarded by default. Set this option to 'random' if you want to randomly pick an 
isogroup to assign a count to.

";

my $t1=shift @ARGV or die $usage;
my $t2=shif

In [22]:
%%bash 

for file in *.sam
do
#strip .fastq and directory s tructure from each file, then
# add suffice .fastq.trim to create output name for each file
results_file="$(basename -a $file).counts"

/Applications/bioinformatics/tag-based_RNAseq/samcount.pl $file \
../../results/20181029-olurida-gonad-assembly_seq2iso.tab \
aligner=bowtie2 mult.iso=random >\
$results_file
done >> bowtiealigner.txt 2>&1

In [23]:
! head bowtiealigner.txt

adding a count to a randomly picked isogroup when a read maps to multiple isogroups
TRINITY_DN15009_c2_g1_i1 has no isogroup designation
TRINITY_DN1226_c0_g1_i6 has no isogroup designation
TRINITY_DN1226_c0_g1_i4 has no isogroup designation
TRINITY_DN1226_c0_g1_i1 has no isogroup designation
TRINITY_DN1226_c0_g1_i5 has no isogroup designation
TRINITY_DN1226_c0_g1_i2 has no isogroup designation
TRINITY_DN15009_c2_g1_i1 has no isogroup designation
TRINITY_DN934_c0_g1_i1 has no isogroup designation
TRINITY_DN934_c0_g1_i2 has no isogroup designation


In [ ]:
! {tagseq}expression_compiler.pl *.sam.counts > quantseq_allcounts.txt

### Dead end ... there is an error in the 20181029-olurida-gonad-assembly_seq2iso.tab file - no second column with isogroup designations so downstream steps don't work.  

## Try alternate isogroup assignment solution ... 

### If you have no assembler-derived isogroups, use cd-hit-est to cluster contigs.
To look for 99% or better matches between contigs taking 30% of length of either longer or shorter sequence.
Downloaded [cd-hit-v4.6.8-2017-1208-source.tar.gz](https://github.com/weizhongli/cdhit/releases) to my bioinf directory, then unpack and compile using this directions (from PDF): 

1. change dir by `cd cd-hit-v4.6.6-2016-0711`
2. compile the programs - I used `make openmp=no` (without multi-threading)
3. `cd cd-hit-auxtools`
4. compile cd-hit-auxtools by `make`
    
Then create variable as shortcut to cdhit directory. 

In [2]:
cdhit = "/Applications/bioinformatics/cd-hit-v4.6.8-2017-1208/"

In [29]:
! {cdhit}cd-hit-est

		====== CD-HIT version 4.7 (built on Nov 17 2018) ======

Usage: /Applications/bioinformatics/cd-hit-v4.6.8-2017-1208/cd-hit-est [Options] 

Options

   -i	input filename in fasta format, required
   -j	input filename in fasta/fastq format for R2 reads if input are paired end (PE) files
 	 -i R1.fq -j R2.fq -o output_R1 -op output_R2 or
 	 -i R1.fa -j R2.fa -o output_R1 -op output_R2 
   -o	output filename, required
   -op	output filename for R2 reads if input are paired end (PE) files
   -c	sequence identity threshold, default 0.9
 	this is the default cd-hit's "global sequence identity" calculated as:
 	number of identical amino acids in alignment
 	divided by the full length of the shorter sequence
   -G	use global sequence identity, default 1
 	if set to 0, then use local sequence identity, calculated as :
 	number of identical amino acids in alignment
 	divided by the length of the alignment
 	NOTE!!! don't use -G 0 unless you use alignment coverage controls
 	see options -aL, -A

### To look for 99% or better matches between contigs taking 30% of length of either longer or shorter sequence:

In [46]:
! {cdhit}cd-hit-est -i trinity/20181029-olurida-gonad-assembly.fasta \
-o trinity/20181029-olurida-gonad-assembly_clust.fasta \
-c 0.99 -G 0 -aL 0.3 -aS 0.3

Program: CD-HIT, V4.7, Nov 17 2018, 15:36:56
Command:
         /Applications/bioinformatics/cd-hit-v4.6.8-2017-1208/cd-hit-est
         -i trinity/20181029-olurida-gonad-assembly.fasta -o
         trinity/20181029-olurida-gonad-assembly_clust.fasta -c
         0.99 -G 0 -aL 0.3 -aS 0.3

Started: Sat Nov 17 16:04:18 2018
                            Output                              
----------------------------------------------------------------
total seq: 181531
longest and shortest : 11860 and 180
Total letters: 154611548
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 178M
Buffer          : 1 X 16M = 16M
Table           : 1 X 19M = 19M
Miscellaneous   : 6M
Total           : 221M

Table limit with the given memory limit:
Max number of representatives: 360313
Max number of word counting entries: 72281583

comparing sequences from          0  to      26884
..........    10000  finished       4207  clusters
..........    20000  finished       913

In [47]:
! head -10 trinity/20181029-olurida-gonad-assembly_clust.fasta.clstr

>Cluster 0
0	11860nt, >TRINITY_DN364_c0_g1... *
1	9929nt, >TRINITY_DN364_c0_g1... at 1:9929:1:9933/+/99.78%
2	11466nt, >TRINITY_DN364_c0_g1... at 1:11291:1:11279/+/99.69%
3	9931nt, >TRINITY_DN364_c0_g1... at 1:9931:1:9933/+/99.59%
>Cluster 1
0	11201nt, >TRINITY_DN364_c0_g1... *
>Cluster 2
0	11110nt, >TRINITY_DN10640_c0_... *
1	10995nt, >TRINITY_DN10640_c0_... at 1:10742:1:10742/+/100.00%


### Adding cluster designations to fasta headers:

In [3]:
cat {tagseq}isogroup_namer.pl

#!/usr/bin/perl

my $usage = "

isogroup_namer.pl:

assigns sequences to \"isogroups\" based on cd-hit-est results, 
outputs:
- a transcriptome with gene=isogroupNNNN identifiers added to headers
- a tab-delimited table of sequences - cluster designations

usage: 
isogroup_namer.pl [fasta file] [cd-hit-est result, .clstr file]

example:
isogroup_namer.pl transcritptome.fasta transcriptome.fasta.clstr 

NOTE: 
run cd-hit-est before this; for example, to look for 99% or better 
matches between contigs taking 30% of their lengths 
cd-hit-est -i transcriptome.fasta -o transcriptome.fasta -c 0.99 -G 0 -aL 0.3 -aS 0.3


";

if ($#ARGV<1) { die $usage;}
open fas, $ARGV[0] or die "cannot open fasta file $ARGV[0]\n\n$usage";
open clus, $ARGV[1] or die "cannot open cd-hit result $ARGV[1]\n\n$usage";

my %iso={};
my $seq;
my $cl;
my @seqs=();

while (<clus>) {
	if ($_=~/^>Cluster (\S+)/) {
		foreach $seq (@seqs) {
			$iso{$seq}=$cl;
		}
#print "isogroup $cl : @seqs\m";

		@seqs=();
	}
	elsif ($_=

In [3]:
cd ../results/

/Users/laura/Documents/roberts-lab/laura-quantseq/results


In [7]:
! {tagseq}isogroup_namer.pl trinity/20181029-olurida-gonad-assembly_clust.fasta \
trinity/20181029-olurida-gonad-assembly_clust.fasta.clstr > \
trinity/20181029-olurida-gonad-assembly_seq2iso.tab

In [8]:
! head trinity/20181029-olurida-gonad-assembly_clust_iso_seq2iso.tab

TRINITY_DN88414_c0_g1_i1	isogroup
TRINITY_DN88428_c0_g1_i1	isogroup
TRINITY_DN88485_c0_g1_i1	isogroup
TRINITY_DN88461_c0_g1_i1	isogroup
TRINITY_DN88458_c0_g1_i1	isogroup
TRINITY_DN88473_c0_g1_i1	isogroup
TRINITY_DN88442_c0_g1_i1	isogroup
TRINITY_DN88493_c0_g1_i1	isogroup
TRINITY_DN88449_c0_g1_i1	isogroup
TRINITY_DN88451_c0_g1_i1	isogroup


In [9]:
! head ../results/trinity/20181029-olurida-gonad-assembly_seq2iso.tab

In [10]:
! head trinity/20181029-olurida-gonad-assembly_clust_iso.fasta

>TRINITY_DN88414_c0_g1_i1 gene=isogroup
GGCACTATTTGAGTCTGATTCTCCTCTTGATAGACGACCATGAGATCGAAATCTCTCAAACAGATCATCAGACCGTTTGCTTTGCATGCGTAAAAATGAATTCGCCCCACAAACAAAGTCCCCGAGGTTTTGTATGATCTGAGTGGAATTCATTTTCTTTTTATTCATGTCTCTGAAGAATTTTTCCATCTTATTGACATCTAACTCCTTCAAAGATGTCAAATCACT
>TRINITY_DN88428_c0_g1_i1 gene=isogroup
CGAAAACAGTCAGATGTAGGTATGGAAATATCCCAATCATCAGAAATCCGACTACAGGTGTAAATGATATTAGTATGTTTACACTGCATGCACCGTTAACATTGACCGATCGACGTGGAGATGGAATGGGAATACGTTCTACTGTCAATGCACACCTTGGAGCTTGATCTGGCAGTGACGAATATTTACTCAAGTAGCAGATAGATAGTTTAATGTTCAGTTTCCGTCACGGGTTTTGTTTCACTCTTACAGAGACCTTGCCCGTGGGGTCCGGGTTGGAATAGGT
>TRINITY_DN88485_c0_g1_i1 gene=isogroup
ATCAAAGGTACGTTGGAGGCAAAGCCTTCCTAAAGCTACCGAGGTCTGACAAAATAAGATGTGTATATTTTAGTGGCTTTGGATCGTTTAATAATTTTTTAAAATGGCCCCTAATTGTGAGCCCCTGGAATGCTGGGGCTGGATTCGCTACTGCTTTTGTATTAACTGATAGTCACTCCCTGTGTATCCACCCCTTTATTGAAGTGTTTCACCACATGGAGTGAGTTCTATAAAGCACTTTGTCACCAGCTTTTGTATATATGTCTACCCCTACTGTGTTGTTGTATGAAATTTTCATATGAATTTAAGGAAGTCTGACTGTGTTGAATTTCAGAAACATGCATGTATCCCTCGAACTGCAGTT

### ARGH. `isogroup_namer.pl` isn't working, does not pull any of the assigned isogroup numbers.

### Try again with the other .fasta (not produced by cd-hit-est): 

In [4]:
! {tagseq}isogroup_namer.pl trinity/20181029-olurida-gonad-assembly.fasta \
trinity/20181029-olurida-gonad-assembly_clust.fasta.clstr > \
trinity/20181029-olurida-gonad-assembly_seq2iso.tab

In [5]:
! head trinity/20181029-olurida-gonad-assembly_iso_seq2iso.tab

TRINITY_DN88414_c0_g1_i1	isogroup
TRINITY_DN88428_c0_g1_i1	isogroup
TRINITY_DN88485_c0_g1_i1	isogroup
TRINITY_DN88461_c0_g1_i1	isogroup
TRINITY_DN88458_c0_g1_i1	isogroup
TRINITY_DN88473_c0_g1_i1	isogroup
TRINITY_DN88442_c0_g1_i1	isogroup
TRINITY_DN88493_c0_g1_i1	isogroup
TRINITY_DN88449_c0_g1_i1	isogroup
TRINITY_DN88451_c0_g1_i1	isogroup


In [6]:
! head ../results/trinity/20181029-olurida-gonad-assembly_seq2iso.tab

In [7]:
! head trinity/20181029-olurida-gonad-assembly_iso.fasta

>TRINITY_DN88414_c0_g1_i1 gene=isogroup
GGCACTATTTGAGTCTGATTCTCCTCTTGATAGACGACCATGAGATCGAAATCTCTCAAACAGATCATCAGACCGTTTGCTTTGCATGCGTAAAAATGAATTCGCCCCACAAACAAAGTCCCCGAGGTTTTGTATGATCTGAGTGGAATTCATTTTCTTTTTATTCATGTCTCTGAAGAATTTTTCCATCTTATTGACATCTAACTCCTTCAAAGATGTCAAATCACT
>TRINITY_DN88428_c0_g1_i1 gene=isogroup
CGAAAACAGTCAGATGTAGGTATGGAAATATCCCAATCATCAGAAATCCGACTACAGGTGTAAATGATATTAGTATGTTTACACTGCATGCACCGTTAACATTGACCGATCGACGTGGAGATGGAATGGGAATACGTTCTACTGTCAATGCACACCTTGGAGCTTGATCTGGCAGTGACGAATATTTACTCAAGTAGCAGATAGATAGTTTAATGTTCAGTTTCCGTCACGGGTTTTGTTTCACTCTTACAGAGACCTTGCCCGTGGGGTCCGGGTTGGAATAGGT
>TRINITY_DN88485_c0_g1_i1 gene=isogroup
ATCAAAGGTACGTTGGAGGCAAAGCCTTCCTAAAGCTACCGAGGTCTGACAAAATAAGATGTGTATATTTTAGTGGCTTTGGATCGTTTAATAATTTTTTAAAATGGCCCCTAATTGTGAGCCCCTGGAATGCTGGGGCTGGATTCGCTACTGCTTTTGTATTAACTGATAGTCACTCCCTGTGTATCCACCCCTTTATTGAAGTGTTTCACCACATGGAGTGAGTTCTATAAAGCACTTTGTCACCAGCTTTTGTATATATGTCTACCCCTACTGTGTTGTTGTATGAAATTTTCATATGAATTTAAGGAAGTCTGACTGTGTTGAATTTCAGAAACATGCATGTATCCCTCGAACTGCAGTT

### The isogroup issue persists. Try another method.  

### How many ">" genes are listed in the original fasta, then in the .clust.fasta produced from cd-hit-est?

In [1]:
! grep -c ">" ../results/trinity/20181029-olurida-gonad-assembly.fasta

181531


In [2]:
! grep -c ">" ../results/trinity/20181029-olurida-gonad-assembly_clust.fasta

143867


### Looks like the .clust.fasta does, indeed, collapse the transcriptome to remove the ton of isoforms. I can just use this .clust.fasta file with my quantseq data to get counts. Question is, do I want to have isoform-specific count data? 

### Download O. lurida genome

In [3]:
! curl http://owl.fish.washington.edu/halfshell/genomic-databank/Olurida_v081.fa > ../references/Olurida_v081.fa

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1090M  100 1090M    0     0  15.3M      0  0:01:11  0:01:11 --:--:-- 19.7MM    0     0  14.9M      0  0:01:12  0:01:05  0:00:07 18.5M01:12  0:01:06  0:00:06 17.5M


In [5]:
# MD5 should = 3ac56372bd62038f264d27eef0883bd3
! md5 ../references/Olurida_v081.fa

MD5 (../references/Olurida_v081.fa) = 3ac56372bd62038f264d27eef0883bd3


## Next steps, once I figure out the isogroup assignment issue

### The below will be used to clean up file locations- will likely need to move other files too. 

In [146]:
! mkdir ../../results/bowtie2-build/

In [153]:
! find ../../results/trinity/ -name '*.bt2' -exec mv {} ../../results/bowtie2-build/ \;

In [154]:
! ls ../../results/bowtie2-build/

20181029-olurida-gonad-assembly.fasta.1.bt2
20181029-olurida-gonad-assembly.fasta.2.bt2
20181029-olurida-gonad-assembly.fasta.3.bt2
20181029-olurida-gonad-assembly.fasta.4.bt2
20181029-olurida-gonad-assembly.fasta.rev.1.bt2
20181029-olurida-gonad-assembly.fasta.rev.2.bt2
